<a href="https://colab.research.google.com/github/Kaua-Rbs/Deep-Learning-From-A-To-Z-With-Pytorch-And-Python/blob/main/Project_3_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Breast Cancer Binary Classification - Parameter Tuning

## Importing Libraries

In [1]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 2.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [3]:
torch.__version__, skorch.__version__, sklearn.__version__

('2.3.0+cu121', '1.0.0', '1.2.2')

## Database

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [6]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [7]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [8]:
previsores.shape

(569, 30)

In [9]:
classe.shape

(569,)

## Neural Network Structure's Class Definition

In [10]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    return X

## Skorch

In [11]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Parameter Tuning

In [13]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16],
          'module__initializer': [torch.nn.init.uniform]}

In [14]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>]}

In [15]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

<ipython-input-10-92800dbb6eed>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-92800dbb6eed>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-92800dbb6eed>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8736  0.2129
      2    24343.2034  0.0460
      3    21607.9790  0.0422
      4    19135.1196  0.0409
      5    16931.4971  0.0419
      6    14986.2345  0.0417
      7    13263.7452  0.0428
      8    11731.9724  0.0439
      9    10367.9490  0.0415
     10     9150.4826  0.0527
     11     8060.7256  0.0422
     12     7080.4180  0.0415
     13     6192.2901  0.0469
     14     5382.8819  0.0422
     15     4640.1386  0.0422
     16     3952.5591  0.0460
     17     3309.9036  0.0510
     18     2702.8657  0.0430
     19     2122.0785  0.0444
     20     1558.6171  0.0425
     21     1003.4027  0.0452
     22      478.4378  0.0433
     23      138.4220  0.0415
     24       90.7287  0.0449
     25       88.3918  0.0560
     26       77.7050  0.0462
     27       70.4366  0.0403
     28       64.3088  0.0442
     29       58.6979  0.0421
     30       54.5392  0.0403
     31       52.0841  0.0398
     32   

<ipython-input-10-92800dbb6eed>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-92800dbb6eed>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-92800dbb6eed>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      5    12690.1400  0.0445
      6    11149.3147  0.0421
      7     9765.4873  0.0431
      8     8523.4345  0.0537
      9     7408.3080  0.0415
     10     6405.5082  0.0420
     11     5502.9294  0.0412
     12     4687.3043  0.0412
     13     3944.7716  0.0486
     14     3263.5476  0.0553
     15     2631.7690  0.0400
     16     2037.1351  0.0400
     17     1466.6355  0.0406
     18      907.2842  0.0443
     19      368.1885  0.0424
     20       70.2290  0.0409
     21       64.8927  0.0446
     22       41.8881  0.0415
     23       38.2924  0.0405
     24       34.4357  0.0391
     25       30.3832  0.0416
     26       28.4990  0.0406
     27       26.7977  0.0416
     28       27.0994  0.0429
     29       26.0352  0.0409
     30       24.2246  0.0405
     31       23.3310  0.0505
     32       23.0972  0.0406
     33       22.5643  0.0408
     34       22.1781  0.0431
     35       21.5842  0.0428
     36       20.8130  0.0572
     37       18.3121  0.0524
     38   

<ipython-input-10-92800dbb6eed>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-92800dbb6eed>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-92800dbb6eed>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


...

      5        2.7880  0.0426
      6        2.7181  0.0476
      7        2.6482  0.0445
      8        2.5785  0.0491
      9        2.5088  0.0472
     10        2.4391  0.0464
     11        2.3696  0.0465
     12        2.3002  0.0434
     13        2.2310  0.0328
     14        2.1619  0.0315
     15        2.0931  0.0323
     16        2.0245  0.0345
     17        1.9562  0.0313
     18        1.8883  0.0318
     19        1.8208  0.0311
     20        1.7538  0.0426
     21        1.6875  0.0415
     22        1.6219  0.0345
     23        1.5571  0.0313
     24        1.4934  0.0318
     25        1.4308  0.0324
     26        1.3697  0.0328
     27        1.3101  0.0322
     28        1.2523  0.0328
     29        1.1966  0.0319
     30        1.1432  0.0311
     31        1.0923  0.0309
     32        1.0442  0.0303
     33        0.9992  0.0322
     34        0.9573  0.0315
     35        0.9187  0.0318
     36        0.8836  0.0331
     37        0.8518  0.0379
     38   

<ipython-input-10-92800dbb6eed>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-92800dbb6eed>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-92800dbb6eed>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      6        3.8201  0.0437
      7        3.7531  0.0363
      8        3.6861  0.0314
      9        3.6191  0.0349
     10        3.5520  0.0314
     11        3.4850  0.0333
     12        3.4180  0.0322
     13        3.3511  0.0318
     14        3.2841  0.0325
     15        3.2171  0.0309
     16        3.1501  0.0330
     17        3.0832  0.0312
     18        3.0163  0.0325
     19        2.9494  0.0311
     20        2.8825  0.0329
     21        2.8156  0.0321
     22        2.7488  0.0329
     23        2.6820  0.0410
     24        2.6153  0.0341
     25        2.5486  0.0322
     26        2.4820  0.0325
     27        2.4155  0.0323
     28        2.3491  0.0330
     29        2.2828  0.0332
     30        2.2167  0.0330
     31        2.1507  0.0416
     32        2.0849  0.0393
     33        2.0194  0.0435
     34        1.9541  0.0425
     35        1.8892  0.0332
     36        1.8247  0.0312
     37        1.7607  0.0331
     38        1.6972  0.0332
     39   

<ipython-input-10-92800dbb6eed>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-92800dbb6eed>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-92800dbb6eed>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      3    51267.8822  0.0858
      4    38467.3745  0.0922
      5    28422.4937  0.0852
      6    20474.0159  0.0862
      7    14073.4524  0.0989
      8     8759.0203  0.0805
      9     4118.9056  0.0838
     10      574.8635  0.0813
     11      151.9170  0.0797
     12      122.1264  0.0834
     13      103.8878  0.0819
     14       87.6194  0.0813
     15       72.8501  0.0808
     16       64.1713  0.0911
     17       63.3306  0.0862
     18       55.2647  0.0858
     19       58.0386  0.0874
     20       58.5939  0.0832
     21       51.1864  0.0835
     22       46.2947  0.0821
     23       43.7537  0.0812
     24       43.3757  0.0825
     25       48.3145  0.0797
     26       53.5200  0.0810
     27       47.2430  0.0914
     28       36.6532  0.0939
     29       38.5515  0.0827
     30       47.1432  0.0939
     31       51.5171  0.0849
     32       53.2163  0.0861
     33       49.0863  0.0811
     34       34.5326  0.0784
     35       33.1561  0.0788
     36   

In [16]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [17]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [18]:
melhor_precisao

0.7944897454904868